In [1]:
import torch
import torch.nn as nn

In [2]:
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3
        # square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 6*6 from image dimension
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        # all dimensions except the batch dimension
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

try:
    data = np.load('mnist_data.npz', allow_pickle=True)
    X_small = data['X_small']
    y_small = data['y_small']
except FileNotFoundError:
    # If the data file does not exist, download and save the data
    X, y = fetch_openml('mnist_784', data_home='./', return_X_y=True)
    X = X / 255
    X_small = X[:1000]
    y_small = y[:1000]
    # Save the data
    np.savez('mnist_data.npz', X_small=X_small, y_small=y_small)

X_small = X_small.reshape((-1, 1, 28,28))
X_small = np.pad(X_small, ([0,0], [0,0], [2,2], [2,2]), mode='constant')
X_training = Variable(torch.from_numpy(X_small.astype('float32')))
y_training = Variable(torch.from_numpy(y_small.astype('int')))

In [4]:
y_training = y_training.long()
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.25)

num_epochs = 10  

for epoch in range(num_epochs):
    outputs = net(X_training)
    loss = criterion(outputs, y_training)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(net.state_dict(), 'lenet_model.pth')

Epoch [1/10], Loss: 2.3027
Epoch [2/10], Loss: 2.3011
Epoch [3/10], Loss: 2.2996
Epoch [4/10], Loss: 2.2981
Epoch [5/10], Loss: 2.2966
Epoch [6/10], Loss: 2.2952
Epoch [7/10], Loss: 2.2937
Epoch [8/10], Loss: 2.2922
Epoch [9/10], Loss: 2.2905
Epoch [10/10], Loss: 2.2888


In [12]:
try:
    data = np.load('mnist_data.npz', allow_pickle=True)
    X_small = data['X_small']
    y_small = data['y_small']
except FileNotFoundError:
    # If the data file does not exist, download and save the data
    X, y = fetch_openml('mnist_784', data_home='./', return_X_y=True)
    X = X / 255
    X_small = X[:1000]
    y_small = y[:1000]
    # Save the data
    np.savez('mnist_data.npz', X_small=X_small, y_small=y_small)

X_small = X_small.reshape((-1, 1, 28,28))
X_small = np.pad(X_small, ([0,0], [0,0], [2,2], [2,2]), mode='constant')
X_test = Variable(torch.from_numpy(X_small.astype('float32')))
y_test = Variable(torch.from_numpy(y_small.astype('int')))


with torch.no_grad():
    net.eval()
    test_outputs = net(X_test)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)

print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 11.60%
